In [ ]:
# Add parent directory to Python path
import os
import sys
from pathlib import Path

# Add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import necessary modules
from strategy.data_processing import load_and_prepare_data, get_clean_data_for_plotting, prepare_rolling_data
from strategy.rolling_strategy import build_rolling_snapshots, filter_snapshots_by_signal
from strategy.visualization import plot_iv_vs_moneyness, plot_snapshot


In [ ]:
df_optimized = load_and_prepare_data('../data/TSLA/TSLA.20200320.csv')
moneyness_clean, iv_clean = get_clean_data_for_plotting(df_optimized)
plot_iv_vs_moneyness(moneyness_clean, iv_clean)


In [ ]:
# Build rolling window strategy snapshots
df_rolling = prepare_rolling_data(df_optimized)
rolling_snapshots = build_rolling_snapshots(df_rolling)

In [ ]:
# Interactive Visualization: Explore Trading Signals Across Time Windows
from ipywidgets import interact, Dropdown, IntSlider, Layout

# Filter snapshots by signal type
snapshot_filters = filter_snapshots_by_signal(rolling_snapshots)
snapshots_with_buy = snapshot_filters['buy']
snapshots_with_sell = snapshot_filters['sell']
snapshots_with_both = snapshot_filters['both']
snapshots_with_any = snapshot_filters['any']


def interactive_visualizer(signal_filter, index):
    """Interactive wrapper that filters snapshots based on signal type"""
    # Get the filtered snapshot list
    if signal_filter == 'All Snapshots':
        filtered_snapshots = list(range(len(rolling_snapshots)))
    elif signal_filter == 'Buy Signals Only':
        filtered_snapshots = snapshots_with_buy
    elif signal_filter == 'Sell Signals Only':
        filtered_snapshots = snapshots_with_sell
    elif signal_filter == 'Both Buy & Sell':
        filtered_snapshots = snapshots_with_both
    elif signal_filter == 'Any Signal':
        filtered_snapshots = snapshots_with_any
    
    if len(filtered_snapshots) == 0:
        print(f"No snapshots match the filter: {signal_filter}")
        return
    
    # Map the index to actual snapshot index
    if index >= len(filtered_snapshots):
        index = len(filtered_snapshots) - 1
    
    actual_snapshot_idx = filtered_snapshots[index]
    plot_snapshot(actual_snapshot_idx, rolling_snapshots)


# Create interactive widgets
signal_dropdown = Dropdown(
    options=['All Snapshots', 'Any Signal', 'Buy Signals Only', 'Sell Signals Only', 'Both Buy & Sell'],
    value='Any Signal',
    description='Filter:',
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)

# Start with 'Any Signal' filter
index_slider = IntSlider(
    min=0,
    max=len(snapshots_with_any)-1 if snapshots_with_any else 0,
    value=0,
    description='Snapshot:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=Layout(width='600px')
)


# Update slider max when filter changes
def update_slider_max(change):
    signal_filter = change['new']
    if signal_filter == 'All Snapshots':
        index_slider.max = len(rolling_snapshots) - 1
    elif signal_filter == 'Buy Signals Only':
        index_slider.max = max(0, len(snapshots_with_buy) - 1)
    elif signal_filter == 'Sell Signals Only':
        index_slider.max = max(0, len(snapshots_with_sell) - 1)
    elif signal_filter == 'Both Buy & Sell':
        index_slider.max = max(0, len(snapshots_with_both) - 1)
    elif signal_filter == 'Any Signal':
        index_slider.max = max(0, len(snapshots_with_any) - 1)
    index_slider.value = 0


signal_dropdown.observe(update_slider_max, names='value')

# Display interactive widget
interact(interactive_visualizer, signal_filter=signal_dropdown, index=index_slider)
